In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
emp_schema="id integer, name string, age integer, salary integer, address string, nominee string"

In [0]:

# due to corrupt record it will fail in FAILFAST
df = spark.read.format("csv") \
  .schema(emp_schema) \
  .option("mode","FAILFAST") \
  .option("header", 'false') \
  .load("/FileStore/tables/new_62.csv")

display(df)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 15) (ip-10-172-209-151.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/FileStore/tables/new_62.csv.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:693)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:662)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:488)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:483)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:82)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:208)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:179)
	at org.apache.spark.scheduler.Task.$anonfun$run$5(Task.scala:142)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:125)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:142)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:97)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:904)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1740)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:907)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:761)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1924)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:103)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:506)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:608)
	... 28 more
Caused by: org.apache.spark.sql.catalyst.util.BadRecordException: org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 3,Pritam,22,150000,Bangalore,India,nominee3
Caused by: org.apache.spark.SparkRuntimeException: [MALFORME

In [0]:

# due to corrupt record it will drop bad records in DROPMALFORMED
df1 = spark.read.format("csv") \
  .schema(emp_schema) \
  .option("mode","DROPMALFORMED") \
  .option("header", 'false') \
  .load("/FileStore/tables/new_62.csv")

display(df1)

id,name,age,salary,address,nominee
1,Manish,26,75000,bihar,nominee1
2,Nikita,23,100000,uttarpradesh,nominee2
5,Vikash,31,300000,null,nominee5


In [0]:
# In permissive, null records will be ignored
df2 = spark.read.format("csv") \
  .schema(emp_schema) \
  .option("mode","PERMISSIVE") \
  .option("header", 'false') \
  .load("/FileStore/tables/new_62.csv")

display(df2)

id,name,age,salary,address,nominee
1,Manish,26,75000,bihar,nominee1
2,Nikita,23,100000,uttarpradesh,nominee2
3,Pritam,22,150000,Bangalore,India
4,Prantosh,17,200000,Kolkata,India
5,Vikash,31,300000,null,nominee5


In [0]:
#lets check which records are corrupt

emp_schema_corrupt="id integer, name string, age integer, salary integer, address string, nominee string, _corrupt_record string"

df2 = spark.read.format("csv") \
  .schema(emp_schema_corrupt) \
  .option("mode","PERMISSIVE") \
  .option("header", 'false') \
  .load("/FileStore/tables/new_62.csv")

display(df2)



id,name,age,salary,address,nominee,_corrupt_record
1,Manish,26,75000,bihar,nominee1,null
2,Nikita,23,100000,uttarpradesh,nominee2,null
3,Pritam,22,150000,Bangalore,India,"3,Pritam,22,150000,Bangalore,India,nominee3"
4,Prantosh,17,200000,Kolkata,India,"4,Prantosh,17,200000,Kolkata,India,nominee4"
5,Vikash,31,300000,null,nominee5,null


In [0]:
#uploading bad records to a path

df2 = spark.read.format("csv") \
  .schema(emp_schema_corrupt) \
  .option("header", 'false') \
  .option("badRecordsPath","/FileStore/tables/emp_badrecord/") \
  .load("/FileStore/tables/new_62.csv")

In [0]:
df2.show()


+---+------+---+------+------------+--------+---------------+
| id|  name|age|salary|     address| nominee|_corrupt_record|
+---+------+---+------+------------+--------+---------------+
|  1|Manish| 26| 75000|       bihar|nominee1|           null|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|           null|
|  5|Vikash| 31|300000|        null|nominee5|           null|
+---+------+---+------+------------+--------+---------------+



In [0]:
dbutils.fs.ls("/FileStore/tables/emp_badrecord/20240110T015623/bad_records/part-00000-95621afe-31bb-4f53-ade0-75b51e687830/")

Out[26]: [FileInfo(path='dbfs:/FileStore/tables/emp_badrecord/20240110T015623/bad_records/part-00000-95621afe-31bb-4f53-ade0-75b51e687830', name='part-00000-95621afe-31bb-4f53-ade0-75b51e687830', size=480, modificationTime=1704851820000)]

In [0]:
bad_records=spark.read.json("/FileStore/tables/emp_badrecord/20240110T015623/bad_records/part-00000-95621afe-31bb-4f53-ade0-75b51e687830/")
bad_records.show(truncate=False)

+---------------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+
|path                             |reason                                                                                                                          |record                                     |
+---------------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+
|dbfs:/FileStore/tables/new_62.csv|org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 3,Pritam,22,150000,Bangalore,India,nominee3|3,Pritam,22,150000,Bangalore,India,nominee3|
|dbfs:/FileStore/tables/new_62.csv|org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 4,Prantosh,17,200000,Kolkata,India,nominee4|